In [67]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
# import nltk
# #this will download nltk in your $HOME/nltk_data directory
# nltk.download("all")
!pip install nltk
#sudo python -m nltk.downloader -d /usr/share/nltk_data all

In [7]:
import pandas as pd

# data preprocessing

def traintestval(dir):
    data = pd.read_csv(dir, header=None)
    data = data.rename(columns = {0:"sentences", 1:"classes"})
    category = pd.get_dummies(data.classes)
    data = pd.concat([data, category], axis=1)
    data = data.drop(columns='classes')
    x = data['sentences'].values
    y = data[['anger',	'fear',	'joy',	'love',	'sadness',	'surprise']].values
    return x,y

x_train, y_train = traintestval('train_lstm.csv')
x_test, y_test = traintestval('test_lstm.csv')
x_val, y_val = traintestval('val_lstmcsv.csv')

print(f"Banyak Data\t: {len(x_train)+len(x_test)+len(x_val)}\nTraining\t: {len(x_train)}\nTesting\t\t: {len(x_test)}\nValidation\t: {len(x_val)}")

Banyak Data	: 20000
Training	: 16000
Testing		: 2000
Validation	: 2000


In [8]:
# validaton set less than 20% from overall dataset, we can merge testing set and validation set, cz we only need data test here

import numpy as np

newx_val = np.concatenate([x_test,x_val])
newy_val = np.concatenate([y_test,y_val]) 
print(f"Banyak Data\t: {len(x_train)+len(x_test)+len(x_val)}\nTraining\t: {len(x_train)}\nValidation\t: {len(newx_val)}\nPresentase\t: {len(newx_val)/(len(x_train)+len(newx_val))*100}%")

Banyak Data	: 20000
Training	: 16000
Validation	: 4000
Presentase	: 20.0%


In [9]:
# case folding

for data in x_train:
    data = data.lower()

for data in newx_val:
    data = data.lower()

x_train, newx_val

(array(['আমি অপমানিত বোধ করি।',
        'আমি খুব নিরাশ বোধ থেকে খুব অভিশপ্ত আশাবাদী হতে পারি শুধুমাত্র যত্নশীল এবং জাগ্রত একজনের আশেপাশে থাকা থেকে',
        'আমি পোস্ট করার জন্য একটি মিনিট ধরছি আমি লোভী ভুল বোধ করছি', ...,
        'আমি সামগ্রিকভাবে শক্তিশালী এবং ভাল বোধ করি',
        'আমি মনে করি এটি একটি অভদ্র মন্তব্য ছিল এবং আমি খুশি যে টি',
        'আমি অনেক কিছু জানি কিন্তু আমি খুব বোকা বোধ করি কারণ আমি এটি চিত্রিত করতে পারি না'],
       dtype=object),
 array(['আমি বরং পচা বোধ করছি তাই আমি এখন খুব উচ্চাভিলাষী নই।',
        'আমি আমার ব্লগ আপডেট করছি কারণ আমি খারাপ বোধ করছি',
        'আমি কখনই তাকে আমার থেকে আলাদা করি না কারণ আমি কখনই চাই না যে সে তার সাথে লজ্জিত বোধ করুক',
        ...,
        'যারা একই জিনিস অনুভব করছেন তাদের জন্য এই তথ্যটি শেয়ার করা আমি গুরুত্বপূর্ণ মনে করি',
        'আমি সত্যিই অনুভব করি যে আপনি যদি কিছু সম্পর্কে যথেষ্ট উত্সাহী হন এবং নিজের প্রতি সত্য থাকেন তবে আপনি সফল হবেন',
        'আমার মনে হচ্ছে আমি অনলাইনে বা এমনকি একটি দেখতে যেকোনো সুন্দর মেক আপ কিনতে চাই

In [10]:
# filtering
from nltk.corpus import stopwords
nltk.download('stopwords')

# collecting english stopword 
stopwords_lists = set(stopwords.words('bengali'))

def filtering(group_of_texts):
    data = []
    for text in group_of_texts:
        data.append([word for word in text if not word in stopwords_lists])
    return data

# removing stopwords
train_without_stopword = filtering(train_token)
val_without_stopword = filtering(val_token)

NameError: name 'nltk' is not defined

In [ ]:
# lemmatization
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

def sentence_lem(sentence):
    words = []
    for word in sentence:
        words.append(WordNetLemmatizer().lemmatize(word))
    return words

def lemmatization(group_of_texts):
    for sentence in group_of_texts:
        sentence = sentence_lem(sentence)
    return group_of_texts

x_train = lemmatization(train_without_stopword)
x_val = lemmatization(val_without_stopword)

In [11]:
# tokenizing
import string
import re
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist 


#removing number
#removing punctuation
#remove whitespace leading & trailing
#remove multiple whitespace into single whitespace
def tokenize(group_of_texts):
    token = []
    for text in group_of_texts:
        text = re.sub(r"\d+", "", text)
        text = text.translate(str.maketrans("","",string.punctuation))
        text = text.strip()
        text = re.sub('\s+',' ',text)
        temp = word_tokenize(text)
        token.append(temp)
    return token

train_token = tokenize(x_train)
val_token = tokenize(newx_val)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\milon\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [12]:
# tokenizing

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=5000, oov_token='-')
tokenizer.fit_on_texts(x_train) 
tokenizer.fit_on_texts(x_val)
 
sequence_train = tokenizer.texts_to_sequences(x_train)
sequence_test = tokenizer.texts_to_sequences(x_val)
 
padded_train = pad_sequences(sequence_train) 
padded_test = pad_sequences(sequence_test)

In [13]:
padded_train, padded_test

(array([[   0,    0,    0, ...,  323,    4, 1867],
        [   0,    0,    0, ...,  913,  234,   34],
        [   0,    0,    0, ...,  298,    4,    9],
        ...,
        [   0,    0,    0, ...,   71,    4,    5],
        [   0,    0,    0, ...,  173,    7, 2134],
        [   0,    0,    0, ...,   12,   23,   10]]),
 array([[   0,    0,    0, ...,  203,   66, 2687],
        [   0,    0,    0, ...,  132,  671,  156],
        [   0,    0,    0, ...,  191,    8,    5],
        ...,
        [   0,    0,    0, ...,  220,   11,    5],
        [   0,    0,    0, ...,   45,  422, 1045],
        [   0,    0,    0, ...,  501, 1041,   41]]))

In [14]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from tensorflow import optimizers

# build up model

model = Sequential([
                    Embedding(input_dim=5000, output_dim=16),
                    LSTM(64),
                    Dense(128, activation='relu'),
                    Dense(64, activation='relu'),
                    Dense(6, activation='softmax')])


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          80000     
                                                                 
 lstm (LSTM)                 (None, 64)                20736     
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dense_2 (Dense)             (None, 6)                 390       
                                                                 
Total params: 117,702
Trainable params: 117,702
Non-trainable params: 0
_________________________________________________________________


In [16]:
from tensorflow import optimizers

# compile mode
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['accuracy'])

In [17]:
from keras.callbacks import EarlyStopping

# membuat early stopping
es = EarlyStopping(monitor = 'val_loss',
                   verbose = 0,
                   patience = 2,
                   mode = 'min',
                   restore_best_weights = False)

# training model

In [20]:
# training model

history = model.fit(padded_train, y_train,epochs = 50,validation_data = (padded_test, newy_val),verbose=2,callbacks=[es])

Epoch 1/50


ValueError: Data cardinality is ambiguous:
  x sizes: 2000
  y sizes: 4000
Make sure all arrays contain the same number of samples.

In [ ]:
# evaluate the model
_, train_acc = model.evaluate(padded_train, y_train, verbose=0)
_, test_acc = model.evaluate(padded_test, newy_val, verbose=0)
print('> Accuracy Train: %.3f' % (train_acc * 100))
print('> Accuracy Test: %.3f' % (test_acc * 100))

In [83]:
import matplotlib.pyplot as plt

# plot loss during training
plt.subplot(121)
plt.title('Loss')
plt.plot(history.history['loss'], label='training')
plt.plot(history.history['val_loss'], label='testing')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.legend()

# plot accuracy during training
plt.subplot(122)
plt.title('Accuracy')
plt.plot(history.history['accuracy'], label='training')
plt.plot(history.history['val_accuracy'], label='testing')
plt.xlabel('Epoch')
plt.legend()
plt.show()